In [1]:
import mlflow
import mlflow.sklearn

In [2]:
#mlflow.start_run()

In [3]:
mlflow.log_param("learning_rate", 0.01)
mlflow.log_param("epochs", 100)

100

In [4]:
mlflow.log_metric("accuracy", 0.85)
mlflow.log_metric("precision", 0.90)

In [23]:
#mlflow.end_run()

In [6]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

In [7]:
import trino
import pandas as pd
import numpy as np

In [8]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [9]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [10]:
conn = trino.dbapi.connect(host='192.168.1.71',port=8083,user='trino',catalog='hive', schema="data") 
cur = conn.cursor() 
cur.execute('show tables') 
i= cur.fetchall() 
i 

[['bankchurners'],
 ['bankloan'],
 ['diabetes'],
 ['german_credit_data'],
 ['gld_price_data'],
 ['heart'],
 ['insurance'],
 ['mobile'],
 ['predictive'],
 ['salary'],
 ['shipping'],
 ['shop_customer'],
 ['tesla'],
 ['walmart_sales']]

In [11]:
conn = trino.dbapi.connect(host='192.168.1.71',port=8083,user='trino',catalog='hive', schema="data") 
cur = conn.cursor() 
cur.execute('select * from tesla') 
f= cur.fetchall() 
f 

[['2010-06-29', 19.0, 25.0, 17.540001, 23.889999, 23.889999, 18766300],
 ['2010-06-30', 25.790001, 30.42, 23.299999, 23.83, 23.83, 17187100],
 ['2010-07-01', 25.0, 25.92, 20.27, 21.959999, 21.959999, 8218800],
 ['2010-07-02', 23.0, 23.1, 18.709999, 19.200001, 19.200001, 5139800],
 ['2010-07-06', 20.0, 20.0, 15.83, 16.110001, 16.110001, 6866900],
 ['2010-07-07', 16.4, 16.629999, 14.98, 15.8, 15.8, 6921700],
 ['2010-07-08', 16.139999, 17.52, 15.57, 17.459999, 17.459999, 7711400],
 ['2010-07-09', 17.58, 17.9, 16.549999, 17.4, 17.4, 4050600],
 ['2010-07-12', 17.950001, 18.07, 17.0, 17.049999, 17.049999, 2202500],
 ['2010-07-13', 17.389999, 18.639999, 16.9, 18.139999, 18.139999, 2680100],
 ['2010-07-14', 17.940001, 20.15, 17.76, 19.84, 19.84, 4195200],
 ['2010-07-15', 19.940001, 21.5, 19.0, 19.889999, 19.889999, 3739800],
 ['2010-07-16',
  20.700001,
  21.299999,
  20.049999,
  20.639999,
  20.639999,
  2621300],
 ['2010-07-19', 21.370001, 22.25, 20.92, 21.91, 21.91, 2486500],
 ['2010-07-20

In [12]:
df=pd.DataFrame(f,columns=[i[0] for i in cur.description])

In [13]:
df

,date,open,high,low,close,adj close,volume
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900
...,...,...,...,...,...,...,...
460,2012-04-25,32.070000,32.990002,32.070000,32.910000,32.910000,712200
461,2012-04-26,32.959999,33.520000,32.910000,33.490002,33.490002,425300
462,2012-04-27,33.599998,33.630001,32.910000,33.340000,33.340000,591000
463,2012-04-30,33.270000,33.360001,32.580002,33.130001,33.130001,413900


In [14]:
df.isnull().sum()

date         0
open         0
high         0
low          0
close        0
adj close    0
volume       0
dtype: int64

In [15]:
df.describe()

,open,high,low,close,adj close,volume
count,465.000000,465.000000,465.000000,465.000000,465.000000,4.650000e+02
mean,26.853097,27.448538,26.180559,26.824215,26.824215,1.344050e+06
std,4.664605,4.747379,4.634086,4.720923,4.720923,1.620229e+06
min,16.139999,16.629999,14.980000,15.800000,15.800000,1.185000e+05
25%,23.260000,23.950001,22.750000,23.219999,23.219999,6.693000e+05
50%,27.010000,27.700001,26.410000,26.959999,26.959999,9.564000e+05
75%,30.000000,30.620001,28.920000,29.840000,29.840000,1.375300e+06
max,38.189999,39.950001,37.110001,38.009998,38.009998,1.876630e+07


In [16]:
x=df.iloc[:,1:5]
x

,open,high,low,close
0,19.000000,25.000000,17.540001,23.889999
1,25.790001,30.420000,23.299999,23.830000
2,25.000000,25.920000,20.270000,21.959999
3,23.000000,23.100000,18.709999,19.200001
4,20.000000,20.000000,15.830000,16.110001
...,...,...,...,...
460,32.070000,32.990002,32.070000,32.910000
461,32.959999,33.520000,32.910000,33.490002
462,33.599998,33.630001,32.910000,33.340000
463,33.270000,33.360001,32.580002,33.130001


In [17]:
y=df.iloc[:,-1]
y

0      18766300
1      17187100
2       8218800
3       5139800
4       6866900
         ...   
460      712200
461      425300
462      591000
463      413900
464      659000
Name: volume, Length: 465, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 2)

In [19]:
model = KNeighborsRegressor()
model.fit(xtrain, ytrain)
print('Training Set Score : ', model.score(xtrain, ytrain))
print('Testing Set Score  : ', model.score(xtest, ytest))

Training Set Score :  0.46763717052544096
Testing Set Score  :  0.16409912056466847


In [20]:
import os 
from random import random, randint 
from mlflow import log_metric, log_param,artifacts


In [21]:
import mlflow

def create_experiment(experiment_name, run_name,model,run_params=None):
    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=run_name):
        if run_params is not None:
            for param in run_params:
                mlflow.log_param(param, run_params[param])

        #for metric in run_metrics:
            #mlflow.log_metric(metric, run_metrics[metric])

       # if confusion_matrix_path is not None:
           # mlflow.log_artifact(confusion_matrix_path, 'confusion_matrix.png')

        #if roc_auc_plot_path is not None:
            #mlflow.log_artifact(roc_auc_plot_path, "roc_auc_plot.png")

        mlflow.set_tag("tag1", "Tesla_Dataset")
        mlflow.set_tags({"tag2":"KNeighborsRegressor"})
        mlflow.sklearn.log_model(model, registered_model_name="Tesla_Dataset")

    print('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))


In [24]:
from random import random, randint
from sklearn.ensemble import RandomForestRegressor

import mlflow
import mlflow.sklearn

with mlflow.start_run(run_name="my_experiment") as run:
    params = {"n_estimators": 5, "random_state": 42}
    sk_learn_rfr = RandomForestRegressor(**params)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_param("param_1", randint(0, 100))
    mlflow.log_metrics({"metric_1": random(), "metric_2": random() + 1})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=sk_learn_rfr,
        artifact_path="KNeighborsRegressor",
        registered_model_name="KNN-random-forest-reg-model",
    )

Registered model 'KNN-random-forest-reg-model' already exists. Creating a new version of this model...
2023/04/11 16:41:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: KNN-random-forest-reg-model, version 3
Created version '3' of model 'KNN-random-forest-reg-model'.


In [25]:
#mlflow.end_run() 

In [26]:
from mlflow.tracking import MlflowClient 

client = MlflowClient() 

experiment_name = "Tesla_Dataset" 

mlflow.get_experiment_by_name(experiment_name) 

mlflow.set_experiment(experiment_name) 


<Experiment: artifact_location='mlflow-artifacts:/944774764477499204', creation_time=1680512956980, experiment_id='944774764477499204', last_update_time=1680512956980, lifecycle_stage='active', name='Tesla_Dataset', tags={}>

In [27]:
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://localhost:5000


In [28]:
mlflow.get_artifact_uri()

'mlflow-artifacts:/944774764477499204/e7eab830c5294df7987d2de5256d1bae/artifacts'

In [29]:
import mlflow.pyfunc

import mlflow.sklearn

model_name = "KNN-random-forest-reg-model"
model_version = 2

model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

y_pred = model.predict(xtest)
print(y_pred)


NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [30]:
from mlflow.tracking import MlflowClient 

client = MlflowClient() 

experiment_name = "Tesla_Dataset" 

mlflow.get_experiment_by_name(experiment_name) 

mlflow.set_experiment(experiment_name) 


<Experiment: artifact_location='mlflow-artifacts:/944774764477499204', creation_time=1680512956980, experiment_id='944774764477499204', last_update_time=1680512956980, lifecycle_stage='active', name='Tesla_Dataset', tags={}>